# Реализация архитектуры Transformers


<img src="https://habrastorage.org/getpro/habr/upload_files/f7a/3a5/e84/f7a3a5e845433fa313070f9c794a5fb7.png" width="750">

### Импортируем необходимые библиотеки

In [1]:
from nltk import WordPunctTokenizer
import numpy as np

### В модель подаются текстовые данные. Будем использовать классику - "Привет, мир!"

In [2]:
inputs = 'Привет мир'

### Входные данные необходимо токенизировать и в качестве примера возьмем базовую токенизацию из библиотеки nltk

In [3]:
tokenzier = WordPunctTokenizer()

In [4]:
tokenzie_seq = tokenzier.tokenize(inputs.lower())

### Получаем набор токенов

In [5]:
tokenzie_seq

['привет', 'мир']

### В оригинальной архитектуре используются эмбеддинги размером 512, но в качестве простоты буду использовать 4

In [6]:
embeddings_dim = 4

### Имитируем создание эмбеддингов

In [7]:
input_embeddings = np.random.rand(len(tokenzie_seq), embeddings_dim)

In [8]:
input_embeddings

array([[0.89700025, 0.74198298, 0.87911062, 0.51858262],
       [0.52839169, 0.53158087, 0.11828162, 0.82088   ]])

### После получения входящих эмбеддингов следует позиционное кодирование, используем классический способ с использованием синуса и косинуса

<img src="https://habrastorage.org/getpro/habr/upload_files/077/39a/c8c/07739ac8ce5c811f974ff3d5150159cc.png" width="750">

In [9]:
def positional_encoding(input_embeddings):
    seq_len, embeddings_dim = input_embeddings.shape

    position = np.arange(seq_len)[:, np.newaxis]
    
    div_term = np.exp(np.arange(0, embeddings_dim, 2) * - (np.log(10000.0) / embeddings_dim))
    
    PE = np.zeros((seq_len, embeddings_dim))
    PE[:, 0::2] = np.sin(position * div_term) 
    
    PE[:, 1::2] = np.cos(position * div_term) 
    
    return PE

In [10]:
input_embeddings += positional_encoding(input_embeddings)

In [11]:
input_embeddings

array([[0.89700025, 1.74198298, 0.87911062, 1.51858262],
       [1.36986267, 1.07188318, 0.12828145, 1.82083   ]])

### Готово! Теперь можем подавать данные в саму модель. 


### Архитектура Transformers делится на 2 основных блока - энкодер и декодер. 

### Первым, что нас встречает в энкодере - многоголовое внимание. Начнем с 2х голов

In [12]:
def create_heads(heads_numbers, heads_n, heads_m):
    WK = []
    WQ = []
    WV = []

    for _ in range(heads_numbers):
        WK_temp = np.random.randint(0, 2, size=(heads_n, heads_m))
        WQ_temp = np.random.randint(0, 2, size=(heads_n, heads_m))
        WV_temp = np.random.randint(0, 2, size=(heads_n, heads_m))

        WK.append(WK_temp)
        WQ.append(WQ_temp)
        WV.append(WV_temp)

    return np.array(WK), np.array(WQ), np.array(WV)

In [13]:
WK, WQ, WV = create_heads(2, 4, 3)

### Получаем матрицы WK, WQ, WV для каждой из головы

In [14]:
WK1, WK2 = WK[0], WK[1]
WQ1, WQ2 = WQ[0], WQ[1]
WV1, WV2 = WV[0], WV[1]

### Теперь необходимо умножить входящую последовательность на каждую из матриц для получения ключей, значений и запросов

### Ключ

In [15]:
K1 = input_embeddings @ WK1
K1

array([[1.77611087, 1.74198298, 4.13967622],
       [1.49814412, 1.07188318, 3.02099463]])

### Значение

In [16]:
V1 = input_embeddings @ WV1
V1

array([[2.39769324, 2.39769324, 1.77611087],
       [1.94911145, 1.94911145, 1.49814412]])

### Запрос

In [17]:
Q1 = input_embeddings @ WQ1
Q1

array([[2.41558287, 0.        , 3.2605656 ],
       [3.19069267, 0.        , 2.89271318]])

### Скалярное произведение запроса и каждого ключа

In [18]:
scores1 = Q1 @ K1.T
scores1

array([[17.78802885, 13.46904244],
       [17.64191988, 13.51898845]])

### Деление на квадратный корень размерности вектора ключа

In [19]:
scores1 = scores1 / np.sqrt(embeddings_dim)
scores1

array([[8.89401442, 6.73452122],
       [8.82095994, 6.75949423]])

### Далее используем softmax для нормализации, чтобы все они были положительны и в сумме равнялись 1.

In [20]:
def softmax(x, axis=1):
    return np.exp(x) / np.sum(np.exp(x), axis=axis, keepdims=True)

In [21]:
scores1 = softmax(scores1)
scores1

array([[0.89655255, 0.10344745],
       [0.88710105, 0.11289895]])

### Вычисление внимания!

In [22]:
attention1 = scores1 @ V1
attention1

array([[2.3512886 , 2.3512886 , 1.74735592],
       [2.34704883, 2.34704883, 1.74472871]])

### Теперь обернем все в одну функцию для одной головы

In [23]:
def attention(input_embeddings, WK, WQ, WV):
    K = input_embeddings @ WK
    Q = input_embeddings @ WQ
    V = input_embeddings @ WV
    
    scores = Q @ K.T
    scores = scores / np.sqrt(embeddings_dim)
    scores = softmax(scores)
    scores = scores @ V
    return scores

### И получаем тоже самое

In [24]:
attention1 = attention(input_embeddings, WK1, WQ1, WV1)
attention1

array([[2.3512886 , 2.3512886 , 1.74735592],
       [2.34704883, 2.34704883, 1.74472871]])

### Вычислим внимание второй головы

In [25]:
attention2 = attention(input_embeddings, WK2, WQ2, WV2)
attention2

array([[4.42414537, 1.10642277, 1.27340492],
       [4.46548777, 1.1493195 , 1.36436631]])

### В конце матрицы внимания конкатенируются

In [26]:
attentions = np.concatenate([attention1, attention2], axis=1)
attentions

array([[2.3512886 , 2.3512886 , 1.74735592, 4.42414537, 1.10642277,
        1.27340492],
       [2.34704883, 2.34704883, 1.74472871, 4.46548777, 1.1493195 ,
        1.36436631]])

### Осталось реализовать функцию многоголового внимания

In [27]:
def multi_head_attention(input_embeddings, heads_numbers, heads_n, heads_m, W_n, W_m):
    WK, WQ, WV = create_heads(heads_numbers, heads_n, heads_m)

    attentions = []
    for i in range(len(WK)):
        WK_cur = WK[i]
        WQ_cur = WQ[i]
        WV_cur = WV[i]

        scores_cur = attention(input_embeddings, WK_cur, WQ_cur, WV_cur)
        attentions.append(scores_cur)

    W = np.random.rand(W_n, W_m)

    return np.concatenate(attentions, axis=1) @ W

In [28]:
Z = multi_head_attention(input_embeddings, 2, 4, 3, 6, 4)

In [29]:
Z

array([[8.8359728 , 3.41152641, 8.93632016, 6.14191246],
       [8.82947139, 3.45527595, 8.93877201, 6.21770766]])

### Далее идет слой Add & Norm, который нормализует значения. (Интересный факт! Если этого слоя бы не было, то на выходе из энкодера в матрице будут nan. Связано это с тем, что значения становятся слишком большими)

In [30]:
def layer_norm(input_embeddings, epsilon=1e-6):
    mean = input_embeddings.mean(axis=-1, keepdims=True)
    std = input_embeddings.std(axis=-1, keepdims=True)
    
    return (input_embeddings - mean) / (std + epsilon)

In [31]:
layer_norm(input_embeddings)

array([[-0.95438355,  1.27230594, -1.0015261 ,  0.6836037 ],
       [ 0.43897274, -0.0416654 , -1.56368617,  1.16637883]])

### Осталось добавить сеть прямого распространения

In [32]:
def relu(x):
    return np.maximum(0, x)

In [33]:
def feed_forward(Z, W1, b1, W2, b2):
    return relu(Z.dot(W1) + b1).dot(W2) + b2

In [34]:
W1 = np.random.randn(4, 8)
W2 = np.random.randn(8, 4)
b1 = np.random.randn(8)
b2 = np.random.randn(4)

In [35]:
output_encoder = feed_forward(Z, W1, b1, W2, b2)
output_encoder

array([[81.40846137, -2.55936503, 38.31852649,  0.66808032],
       [82.04942637, -2.51340001, 38.87532749,  0.67055369]])

In [36]:
output_encoder = layer_norm(output_encoder)
output_encoder

array([[ 1.52671779, -0.94096884,  0.26037018, -0.84611913],
       [ 1.52431641, -0.94131278,  0.26547345, -0.84847708]])

### Теперь соберем энкодер воедино!

In [37]:
def encoder_layer(input_embeddings, heads_numbers, heads_n, heads_m, W_n, W_m):
    Z = multi_head_attention(input_embeddings, heads_numbers, heads_n, heads_m, W_n, W_m)

    W1 = np.random.randn(4, 8)
    W2 = np.random.randn(8, 4)
    b1 = np.random.randn(8)
    b2 = np.random.randn(4)

    output = feed_forward(Z, W1, b1, W2, b2)

    return layer_norm(output, Z)

In [38]:
output_encoder = encoder_layer(input_embeddings, 2, 4, 3, 6, 4)
output_encoder

array([[ 1.35783162, -1.11614541, -0.11406513, -0.09956317],
       [ 1.35763904, -1.11612881, -0.11425058, -0.09921788]])

### В классической архитектуре использовалось 6 слоев энкодера, поэтому сделаем тоже самое

In [39]:
def encoder(input_embeddings, n=6):
    for _ in range(n):
        input_embeddings = encoder_layer(input_embeddings, 2, 4, 3, 6, 4)

    return input_embeddings

In [40]:
output_encoder = encoder(input_embeddings)
output_encoder

array([[-1.94001839,  0.60686808,  1.91929143, -0.37666407],
       [-1.94001839,  0.60686808,  1.91929143, -0.37666407]])

### Начало декодера - выходные эмбеддинги, но в качестве первого токена на вход этому слою подается специальный токен sos, который означает начало последовательности (start of the sequence)

In [41]:
sos_embedding = np.random.rand(len(tokenzie_seq), embeddings_dim) 

### Первая часть идентична энкодеру - многоголовое внимание и нормализация

In [42]:
sos_embedding += positional_encoding(sos_embedding)
sos_embedding

array([[0.65295661, 1.96540852, 0.52657657, 1.81403241],
       [1.33200423, 1.53020025, 0.58657478, 1.48053814]])

In [43]:
decoder_self_attention = multi_head_attention(sos_embedding, 2, 4, 3, 6, 4)

In [44]:
decoder_self_attention

array([[7.64862277, 5.90350643, 6.78472443, 5.96427115],
       [7.63867652, 5.91584367, 6.78332633, 5.97368071]])

In [45]:
decoder_self_attention = layer_norm(decoder_self_attention + sos_embedding)
decoder_self_attention

array([[ 1.3834969 ,  0.1532345 , -1.4323164 , -0.104415  ],
       [ 1.72996616, -0.54288109, -0.65639073, -0.53069434]])

### Самое интересное начинается здесь! Это внимание энкодера-декодера, куда поступают выходы из энкодера. Ключевое отличие от собственного внимания - ключ и значение получаются из выхода энкодера, а запрос формируется из поступающих данных от декодера

In [46]:
def encoder_decoder_attention(encoder_output, attention_input, WQ, WK, WV):
    K = encoder_output @ WK    
    V = encoder_output @ WV    
    Q = attention_input @ WQ   

    scores = Q @ K.T
    scores = scores / np.sqrt(embeddings_dim)
    scores = softmax(scores)
    scores = scores @ V
    return scores

In [47]:
def multi_head_encoder_decoder_attention(encoder_output, attention_input, heads_numbers, heads_n, heads_m, W_n, W_m):
    WK, WQ, WV = create_heads(heads_numbers, heads_n, heads_m)

    attentions = []
    for i in range(len(WK)):
        WK_cur = WK[i]
        WQ_cur = WQ[i]
        WV_cur = WV[i]
        
        scores_cur = encoder_decoder_attention(encoder_output, attention_input, WK_cur, WQ_cur, WV_cur)
        attentions.append(scores_cur)
    
    W = np.random.rand(W_n, W_m)
    concatenated_attention = np.concatenate(attentions, axis=1) @ W

    return concatenated_attention


In [48]:
Z_encoder_decoder = multi_head_encoder_decoder_attention(output_encoder, decoder_self_attention, 2, 4, 3, 6, 4)
Z_encoder_decoder

array([[3.07226084, 1.68657209, 1.5382457 , 2.50516641],
       [3.07226084, 1.68657209, 1.5382457 , 2.50516641]])

### Теперь сформируем слой декодера

In [49]:
def decoder_layer(decoder_inputs, encoder_outputs, heads_numbers, heads_n, heads_m, W_n, W_m):
    Z = multi_head_attention(decoder_inputs, heads_numbers, heads_n, heads_m, W_n, W_m)

    W1 = np.random.randn(4, 8)
    W2 = np.random.randn(8, 4)
    b1 = np.random.randn(8)
    b2 = np.random.randn(4)

    output = feed_forward(Z, W1, b1, W2, b2)

    output = layer_norm(output, Z)

    Z_encoder_decoder = multi_head_encoder_decoder_attention(encoder_outputs, decoder_self_attention, 2, 4, 3, 6, 4)

    Z_encoder_decoder = layer_norm(Z_encoder_decoder + Z)

    output = feed_forward(Z_encoder_decoder, W1, b1, W2, b2)
    return layer_norm(output + Z_encoder_decoder)

In [50]:
output_decoder = decoder_layer(sos_embedding, output_encoder, 2, 4, 3, 6, 4)
output_decoder

array([[-0.48526621, -1.22226021,  1.49124699,  0.21627943],
       [-0.48534158, -1.22136576,  1.49221279,  0.21449455]])

### Аналогично с энкодером - декодер имеет 6 частей

In [51]:
def decoder(decoder_input, output_encoder, n=6):
    for _ in range(n):
        decoder_input = decoder_layer(decoder_input, output_encoder, 2, 4, 3, 6, 4)

    return decoder_input

In [52]:
decoder_outputs = decoder(sos_embedding, output_decoder)
decoder_outputs

array([[-1.14222914,  1.47216652, -0.65159337,  0.32165598],
       [-1.14222914,  1.47216654, -0.65159334,  0.32165593]])

### На выходе нам необходимо добавить линейный слой, чтобы преобразовать выходные данные в вектор размера словаря

In [53]:
def linear(x, W, b):
    return np.dot(x, W) + b

### А затем получаем вероятности токенов с помощью функции активации softmax

In [54]:
def output_probabilities(decoder_outputs, embeddings_dim, vocabulary_length):
    for decoder_output in decoder_outputs:
        linear_output = linear(decoder_output, np.random.randn(embeddings_dim, vocabulary_length), np.random.randn(vocabulary_length))

        softmax_output = softmax(linear_output, axis=0)

        print(softmax_output)

In [55]:
output_probabilities(decoder_outputs, embeddings_dim, 4)

[0.13389328 0.37876943 0.33216612 0.15517117]
[0.42477685 0.00193619 0.18435459 0.38893238]


In [56]:
class Embeddings:
    def __init__(self, embeddings_dim):
        self.embeddings_dim = embeddings_dim # длина эмбеддингов

        self.words_list_ru = ['<sos>', 'привет', 'мир', '<eos>', '<unk>'] # словарь первого языка 
        self.words_list_en = ['<sos>', 'hello', 'world', '<eos>', '<unk>'] # словарь второго языка

        self.create_input_ids_dict() # создаем словарь id токенов
        self.create_embeddings() # создаем "матрицу" эмбеддингов


    def get_vocab_length(self):
        return len(self.words_list_ru) # возвращаем длину словаря (необходимо для softmax по выходу из декодера)


    def create_input_ids_dict(self):
        self.input_ids_dict = {}
        for i in range(len(self.words_list_ru)):
            self.input_ids_dict[i] = self.words_list_ru[i] # формируем словарь id токенов


    def get_input_id(self, token):
        return list(self.input_ids_dict.keys())[list(self.input_ids_dict.values()).index(token)] # возвращаем id по токену


    def get_input_ids(self, tokens):
        input_ids = []
        for token in tokens:
            input_ids.append(self.get_input_id(token))
        return input_ids # возвращаем все id токенов входной последовательности


    def create_embeddings(self):
        self.embeddings_dict = {}
        for i in range(len(self.words_list_ru)):
            self.embeddings_dict[i] = np.random.rand(self.embeddings_dim) # создаем "матрицу" эмбеддингов


    def get_embeddings(self, input_ids):
        embeddings = []
        for input_id in input_ids:
            embeddings.append(self.embeddings_dict[input_id])
        return np.array(embeddings) # возвращаем эмбеддинги


    def __call__(self, tokens):
        input_ids = self.get_input_ids(tokens)
        return self.get_embeddings(input_ids)
    

    def get_translation(self, input_id):
        return self.words_list_en[input_id] # получаем перевод

In [57]:
class SimpleTransformer():
    def __init__(self, 
                 embeddings_dim=4,
                 heads_numbers=2,
                 heads_n=4,
                 heads_m=3,
                 W_n=6,
                 W_m=4):
        self.tokenizer = WordPunctTokenizer()
        self.embeddings_dim = embeddings_dim
        self.embeddings = Embeddings(self.embeddings_dim)

        self.heads_numbers = heads_numbers
        self.head_n = heads_n
        self.head_m = heads_m
        self.W_n = W_n
        self.W_m = W_m
        self.vocab_length = self.embeddings.get_vocab_length()
        

    def __call__(self, inputs):
        self.tokenized_seq = self.tokenizer.tokenize(inputs.lower())
        self.input_embeddings = self.embeddings(self.tokenized_seq)

        # пропускаем через энкодер
        encoder_output = self.encoder(self.input_embeddings)
        
        # пропускаем через декодер
        decoder_output = self.decoder(encoder_output)
        
        # получаем вероятности предсказания
        probabilities = self.output_probabilities(decoder_output)

        # получаем наиболее вероятные токены и вовзращаем их
        translation = self.translate(probabilities)
        
        return translation


    def translate(self, probabilities):
        predicted_ids = np.argmax(probabilities, axis=1) # получаем id наиболее вероятных токены последовательности

        translated_tokens = [self.embeddings.get_translation(idx) for idx in predicted_ids] # ищем их в словаре

        return " ".join(translated_tokens)

    # далее идут функции, что использовать ранее, поэтому нет особого смысла их комментировать

    def positional_encoding(self, input_embeddings):
        seq_len, embeddings_dim = input_embeddings.shape

        position = np.arange(seq_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, embeddings_dim, 2) * -(np.log(10000.0) / embeddings_dim))

        PE = np.zeros((seq_len, embeddings_dim))
        PE[:, 0::2] = np.sin(position * div_term) 
        PE[:, 1::2] = np.cos(position * div_term) 
        
        return PE
    
    def create_heads(self):
        WK = []
        WQ = []
        WV = []

        for _ in range(self.heads_numbers):
            WK_temp = np.random.randn(self.head_n, self.head_m)
            WQ_temp = np.random.randn(self.head_n, self.head_m)
            WV_temp = np.random.randn(self.head_n, self.head_m)

            WK.append(WK_temp)
            WQ.append(WQ_temp)
            WV.append(WV_temp)

        return np.array(WK), np.array(WQ), np.array(WV)
    

    def softmax(self, x, axis=1):
        return np.exp(x) / np.sum(np.exp(x), axis=axis, keepdims=True)


    def attention(self, input_embeddings, WK, WQ, WV):
        K = input_embeddings @ WK
        Q = input_embeddings @ WQ
        V = input_embeddings @ WV
        
        scores = Q @ K.T
        scores = scores / np.sqrt(self.embeddings_dim)
        scores = self.softmax(scores)
        scores = scores @ V
        return scores
    

    def multi_head_attention(self, input_embeddings):
        WK, WQ, WV = self.create_heads()

        attentions = []
        for i in range(len(WK)):
            WK_cur = WK[i]
            WQ_cur = WQ[i]
            WV_cur = WV[i]

            scores_cur = self.attention(input_embeddings, WK_cur, WQ_cur, WV_cur)
            attentions.append(scores_cur)

        W = np.random.rand(self.W_n, self.W_m)

        return np.concatenate(attentions, axis=1) @ W
    

    def layer_norm(self, input_embeddings, epsilon=1e-6):
        mean = input_embeddings.mean(axis=-1, keepdims=True)
        std = input_embeddings.std(axis=-1, keepdims=True)
        
        return (input_embeddings - mean) / (std + epsilon)


    def relu(self, x):
        return np.maximum(0, x)
    

    def feed_forward(self, Z, W1, b1, W2, b2):
        return self.relu(Z @ W1 + b1) @ W2 + b2


    def encoder_layer(self, input_embeddings):
        Z = self.multi_head_attention(input_embeddings)

        W1 = np.random.randn(self.embeddings_dim, 8)
        W2 = np.random.randn(8, self.embeddings_dim)
        b1 = np.random.randn(8)
        b2 = np.random.randn(self.embeddings_dim)

        output = self.feed_forward(Z, W1, b1, W2, b2)

        return self.layer_norm(output + Z)
    

    def encoder(self, input_embeddings, n=6):
        for _ in range(n):
            input_embeddings = self.encoder_layer(input_embeddings)

        return input_embeddings


    def encoder_decoder_attention(self, encoder_output, decoder_input, WK, WQ, WV):
        K = encoder_output @ WK    
        V = encoder_output @ WV    
        Q = decoder_input @ WQ   

        scores = Q @ K.T
        scores = scores / np.sqrt(self.embeddings_dim)
        scores = self.softmax(scores)
        scores = scores @ V
        return scores


    def multi_head_encoder_decoder_attention(self, encoder_output, decoder_input):
        WK, WQ, WV = self.create_heads()

        attentions = []
        for i in range(len(WK)):
            WK_cur = WK[i]
            WQ_cur = WQ[i]
            WV_cur = WV[i]
            
            scores_cur = self.encoder_decoder_attention(encoder_output, decoder_input, WK_cur, WQ_cur, WV_cur)
            attentions.append(scores_cur)
        
        W = np.random.rand(self.W_n, self.W_m)
        concatenated_attention = np.concatenate(attentions, axis=1) @ W

        return concatenated_attention


    def decoder_layer(self, decoder_inputs, encoder_outputs):
        Z = self.multi_head_attention(decoder_inputs)

        Z_encoder_decoder = self.multi_head_encoder_decoder_attention(encoder_outputs, Z)

        W1 = np.random.randn(self.embeddings_dim, 8)
        W2 = np.random.randn(8, self.embeddings_dim)
        b1 = np.random.randn(8)
        b2 = np.random.randn(self.embeddings_dim)

        output = self.feed_forward(Z_encoder_decoder, W1, b1, W2, b2)

        return self.layer_norm(output + Z_encoder_decoder)


    def decoder(self, encoder_output, n=6):
        decoder_output = np.random.rand(len(self.tokenized_seq), self.embeddings_dim)  
        
        for _ in range(n):
            decoder_output = self.decoder_layer(decoder_output, encoder_output)

        return decoder_output
    

    def linear(self, x, W, b):
        return np.dot(x, W) + b


    def output_probabilities(self, decoder_outputs):
        probabilities = []
        for decoder_output in decoder_outputs:
            linear_output = self.linear(decoder_output, np.random.randn(self.embeddings_dim, self.vocab_length), np.random.randn(self.vocab_length))
            softmax_output = self.softmax(linear_output, axis=0)
            probabilities.append(softmax_output)

        return np.array(probabilities)

### Создаем класс переводчика

In [58]:
transformer =  SimpleTransformer()

### Проверяем работу!

In [81]:
inputs = 'Привет мир'
transformer_outputs = transformer(inputs)

In [82]:
transformer_outputs

'<sos> hello'

### Благодаря собственной удаче (т.к. все мы рандомизировали и не обучали) получаем начало последовательности и hello!